# Suite2p notebook
This file is read only. Please make a copy in your home directory before modifying this file.

In [2]:
import sys
sys.path.insert(1, './extras')

from parse_xml import parse_xml
from raw2h5 import raw2h5
import logging
import os
import glob
import argparse

## Setup input
Setting the ```run_mode``` variable will control which parts of the pipeline are run.
- all: Run the entire pipeline
- suite2p: Run only the registration/cell extraction part
- deconv: Run the deconvolution algorithm from the MATLAB version of suite2p on the extracted cells

Either insert the path to a file in the ```params_file``` variable, or fill your entries in the db variable below.
Make sure your input/output paths are created before running this.

In [6]:
logging.basicConfig(format="%(asctime)s %(relativeCreated)12d [%(filename)s:%(funcName)15s():%(lineno)s] [%(process)d] %(message)s", 
                    datefmt='%Y-%m-%d %H:%M:%S', level=logging.DEBUG)
run_mode = 'all' # Use 'suite2p' to just run the registration/cell extraction, 'deconv' for just the deconvolution or 'all' for everything
params_file = '' # Place path to params.py here if you want to use a separate file.
if os.path.isfile(params_file):
    with open(params_file) as f:
        code = compile(f.read(), params_file, 'exec')
        exec(code)
else:
    db = []
    
    db.append({
        "mouse_name": "183628",
        "date": "2019-06-11",
        "expts": ['1_001'],
        "raw_dir": "//mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/",
        "tif_dir": "E:/Users/samsoon.inayat/S_Drive/Tif_Data/",
        "results_dir": "E:/Users/samsoon.inayat/S_Drive/Processed_Data",
        "ops": {
            'do_bidiphase': True,
            'bidiphase': 1,
            'bidi_corrected': True,
            'sparse_mode': False,
            'diameter': 15
        }
    })

## Run the actual pipeline

In [10]:
for i in db:
    comb_dir = '_'.join(str(x) for x in i['expts'])
    tif_dir = os.path.join(i['tif_dir'], i['mouse_name'], i['date'], comb_dir)
    results_dir = os.path.join(i['results_dir'], i['mouse_name'], i['date'], comb_dir)
    
    if not os.path.isdir(results_dir):
        logging.info(f'Creating results directory {results_dir}')
        os.makedirs(results_dir)
    
    if 'abf_dir' in i:
        abf_dir = [ os.path.join(i['abf_dir'], i['mouse_name'], i['date'] + "_" + str(x) + ".abf") for x in i['expts'] ]
    else:
        abf_dir = None
    print(f'Running expt {i}')
    print(f'Temp directory: {tif_dir}')
    print(f'Results directory: {results_dir}')
    i['in_dir'] = os.path.join(i['raw_dir'], i['mouse_name'], i['date'])
    offset = 1
    nframes = []
    for j in i['expts']:
        in_dir = os.path.join(i['in_dir'], str(j))
        
        xml_path = os.path.join(in_dir, "Experiment.xml")
        logging.info(f'Parsing Experiment.xml in {xml_path}')
        if os.path.exists(xml_path):
            (x, y, fr, num_frames, num_planes) = parse_xml(xml_path)
            nframes.append(num_frames)
        else:
            raise ValueError(f"Experiment.xml does not exist at {xml_path}")
            
    if run_mode == 'suite2p' or run_mode == 'all':
        from run_suite2p import run_suite2p
        ops = run_suite2p(i, results_dir, fr/num_planes, num_planes)
    if run_mode == 'deconv' or run_mode == 'all':
        from deconv import do_deconv
        do_deconv(results_dir, num_planes/fr, nframes, i['expts'], abf_dir, nplanes=num_planes)

2021-02-11 23:00:59      6952304 [<ipython-input-10-364e49129f17>:       <module>():24] [1744] Parsing Experiment.xml in //mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/183628\2019-06-11\1_001\Experiment.xml


Running expt {'mouse_name': '183628', 'date': '2019-06-11', 'expts': ['1_001'], 'raw_dir': '//mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/', 'tif_dir': 'E:/Users/samsoon.inayat/S_Drive/Tif_Data/', 'results_dir': 'E:/Users/samsoon.inayat/S_Drive/Processed_Data', 'ops': {'do_bidiphase': True, 'bidiphase': 1, 'bidi_corrected': True, 'sparse_mode': False, 'diameter': 15}, 'in_dir': '//mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/183628\\2019-06-11'}
Temp directory: E:/Users/samsoon.inayat/S_Drive/Tif_Data/183628\2019-06-11\1_001
Results directory: E:/Users/samsoon.inayat/S_Drive/Processed_Data\183628\2019-06-11\1_001
{'suite2p_version': '0.9.3', 'look_one_level_down': True, 'fast_disk': [], 'delete_bin': False, 'mesoscan': False, 'bruker': False, 'h5py': [], 'h5py_key': 'data', 'save_path0': 'E:/Users/samsoon.inayat/S_Drive/Processed_Data\\183628\\2019-06-11\\1_001', 'save_folder': [], 'subfolders': [], 'move_bin': False, 'nplanes': 1, 'nchannels': 1, 'functional

E:\Users\samsoon.inayat\Anaconda3\envs\suite2p\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Plane 0 processed in 503.99 sec (can open in GUI).
total = 504.84 sec.
TOTAL RUNTIME 504.84 sec


AttributeError: type object 'skimage.transform._hough_transform.array' has no attribute '__reduce_cython__'